In [1]:
kdprov = '76'

In [2]:
# Parameters
kdprov = "21"


In [3]:
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.features as features
import numpy as np
from rasterio.enums import Resampling
from rasterio.crs import CRS
import itertools

from shapely.geometry import box

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [4]:
from glob import glob

In [5]:
data_popdens_100m=glob("data/output/1_population_density/grid_pop_"+kdprov+"*_100m.gpkg")
data_popdens_100m=[i for i in data_popdens_100m if len(i)==56]

In [6]:
data_popdens_100m

['data/output/1_population_density/grid_pop_2102_100m.gpkg',
 'data/output/1_population_density/grid_pop_2171_100m.gpkg',
 'data/output/1_population_density/grid_pop_2172_100m.gpkg',
 'data/output/1_population_density/grid_pop_2101_100m.gpkg',
 'data/output/1_population_density/grid_pop_2104_100m.gpkg',
 'data/output/1_population_density/grid_pop_2105_100m.gpkg',
 'data/output/1_population_density/grid_pop_2103_100m.gpkg']

In [7]:
from rasterio.enums import Resampling
import os
def resampling_1km(dt):
    global kdprov
    kdkab=dt.split('/')[3].split('_')[2]
    if os.path.isfile('data/output/1_population_density/grid_pop_'+kdkab+'_1km.tif')==False:
        popdens_100m=gpd.read_file(dt)
        popdens_100m['j_pend_round']=popdens_100m.j_pend.apply(lambda y: np.round(y))
        raster = rasterio.open(r"/vsigs/bps-gcp-bucket/Degree of Urbanisation/GHSL data/2020/prov/2020_"+kdprov+".tif")
        geom_value = ((geom,value) for geom, value in zip(popdens_100m.geometry, popdens_100m['j_pend_round']))
        rasterized = features.rasterize(geom_value,
                                out_shape = raster.shape,
                                transform = raster.transform,
                                all_touched = False,
                                fill = 0,
                                dtype = np.float64)
        with rasterio.open(
            "data/temp/rasterized_popdens_"+kdkab+"_100.tif", "w",
            driver = "GTiff",
            transform = raster.transform,
            dtype = np.float64,
            count = 1,
            width = raster.width,
            height = raster.height) as dst:
            dst.write(rasterized, indexes = 1)
    
        target_res = (1000, 1000)

        with rasterio.open("data/temp/rasterized_popdens_"+kdkab+"_100.tif") as src:
            data, transform = rasterio.warp.reproject(source=src.read(),
                                src_transform=src.transform,
                                src_crs=CRS.from_string("ESRI:54009"),
                                dst_crs=CRS.from_string("ESRI:54009"),
                                dst_nodata=0,
                                dst_resolution=target_res,
                                              #dtype = np.float64
                                resampling=Resampling.sum)
            profile = src.profile
            profile.update(transform=transform, driver='GTiff',
                    height=data.shape[1], width=data.shape[2])

            with rasterio.open('data/output/1_population_density/grid_pop_'+kdkab+'_1km.tif', 'w', **profile) as dst:
                dst.write(data)
    else:
        print("skip")
    if (os.path.isfile("data/output/1_population_density/grid_pop_"+kdkab+"_1km_w_bu.gpkg")==False):
        with rasterio.open('data/output/1_population_density/grid_pop_'+kdkab+'_1km.tif') as dataset:
            data = dataset.read(1)

            t = dataset.transform
            move_x = t[0]
            move_y = t[4]
            height = dataset.height
            width = dataset.width 
    
            polygons = []
            indices = list(itertools.product(range(width), range(height)))
            for x,y in indices:
                x_min, y_max = t * (x,y)
                x_max = x_min + move_x
                y_min = y_max + move_y
                polygons.append(box(x_min, y_min, x_max, y_max))
        
        data_list = []
        for x,y in indices:
            data_list.append(data[y,x])
        vect_tmp = gpd.GeoDataFrame(data=data_list, geometry=polygons, columns=['DN'])
        vect_tmp.crs='ESRI:54009'
        vect_tmp.DN=vect_tmp.DN.apply(lambda y: 0 if y<=0 else y)
        y=gpd.GeoDataFrame()
        data_builtup=gpd.read_file("data/input/GHSL Data/2020/gpkg/ghs_built_"+kdkab+"_nonzero.shp")#.to_crs(('ESRI:54009'))
        data_builtup['builtup']=data_builtup['value']
        vect_=vect_tmp.reset_index()
        vect_['index_o']=vect_['index'].apply(lambda y:str(y).zfill(7))
        data_bu_per1km=vect_.sjoin(data_builtup,how='inner'
                          ).sort_values('index_o')[['index_o','builtup']].groupby('index_o').agg('sum').reset_index()
        vect_bu=vect_.merge(data_bu_per1km,how='inner')
        vect_bu['p_builtup']=vect_bu.builtup/1e6
        vect_bu['pop']=vect_bu.DN
        vect_bu=vect_bu[['geometry','p_builtup','pop']].fillna(0)
        vect_bu.crs='ESRI:54009'
        vector = gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('idkab==@kdkab').to_crs("ESRI:54009")
        vect_bu['index_o']=vect_bu.index.astype(str).str.zfill(8)
        vect_bu.sjoin(vector)[['geometry','p_builtup','pop','index_o']].dissolve(by=['index_o']).to_file("data/output/1_population_density/grid_pop_"+kdkab+"_1km_w_bu.gpkg",driver='GPKG')
    else:
        print("skip")

In [8]:
from tqdm.contrib.concurrent import process_map
process_map(resampling_1km,data_popdens_100m)   

  0%|          | 0/7 [00:00<?, ?it/s]

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/geopandas/geodataframe.py:2072: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: ESRI:54009
Right CRS: None

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


[None, None, None, None, None, None, None]